In [10]:
from edward.models import Normal
import tensorflow as tf
import numpy as np

def neural_network(x):
    h = tf.tanh(tf.matmul(x, W_0) + b_0)
    h = tf.tanh(tf.matmul(h, W_1) + b_1)
    h = tf.matmul(h, W_2) + b_2
    return tf.reshape(h, [-1])

In [11]:
def build_toy_dataset(N, noise_std=0.5):
    X = np.concatenate([np.linspace(0, 2, num=N / 2),
                       np.linspace(6, 8, num=N / 2)])
    y = 2.0 * X + 10 * np.random.normal(0, noise_std, size=N)
    X = X.astype(np.float32).reshape((N, 1))
    y = y.astype(np.float32)
    return X, y

In [12]:
N = 40
D = 1

X_train, y_train = build_toy_dataset(N)
X_test, y_test = build_toy_dataset(N)

W_0 = Normal(mu=tf.zeros([D, 10]), sigma=tf.ones([D, 10]))
W_1 = Normal(mu=tf.zeros([10, 10]), sigma=tf.ones([10, 10]))
W_2 = Normal(mu=tf.zeros([10, 1]), sigma=tf.ones([10, 1]))
b_0 = Normal(mu=tf.zeros([10]), sigma=tf.ones(10))
b_1 = Normal(mu=tf.zeros([10]), sigma=tf.ones(10))
b_2 = Normal(mu=tf.zeros([1]), sigma=tf.ones(1))

x = X_train
y = Normal(mu=neural_network(x), sigma=0.1 * tf.ones(N))

# Inference

In [17]:
qW_0 = Normal(mu=tf.Variable(tf.random_normal([D, 10])),
             sigma=tf.nn.softplus(tf.Variable(tf.random_normal([D, 10]))))
qW_1 = Normal(mu=tf.Variable(tf.random_normal([10, 10])),
             sigma=tf.nn.softplus(tf.Variable(tf.random_normal([10, 10]))))
qW_2 = Normal(mu=tf.Variable(tf.random_normal([10, 1])),
             sigma=tf.nn.softplus(tf.Variable(tf.random_normal([10, 1]))))
qb_0 = Normal(mu=tf.Variable(tf.random_normal([10])),
             sigma=tf.nn.softplus(tf.Variable(tf.random_normal([10]))))
qb_1 = Normal(mu=tf.Variable(tf.random_normal([10])),
             sigma=tf.nn.softplus(tf.Variable(tf.random_normal([10]))))
qb_2 = Normal(mu=tf.Variable(tf.random_normal([1])),
             sigma=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))

In [35]:
import edward as ed

inference = ed.KLqp({W_0: qW_0, b_0:qb_0,
                    W_1: qW_1, b_1:qb_1,
                    W_2: qW_2, b_2:qb_2}, data={y: y_train})
inference.run(n_iter=5000)

Iteration    1 [  0%]: Loss = 350572.781
Iteration  500 [ 10%]: Loss = 36384.062
Iteration 1000 [ 20%]: Loss = 34581.926
Iteration 1500 [ 30%]: Loss = 35923.652
Iteration 2000 [ 40%]: Loss = 33983.094
Iteration 2500 [ 50%]: Loss = 32668.848
Iteration 3000 [ 60%]: Loss = 32797.055
Iteration 3500 [ 70%]: Loss = 32770.121
Iteration 4000 [ 80%]: Loss = 32834.484
Iteration 4500 [ 90%]: Loss = 32692.566
Iteration 5000 [100%]: Loss = 32966.164
